# Mesogens with NP | Temperature Ramps

## Temperature: 7.2 -> 7.4 | CPU Run

### Date: 12/03/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [6]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [7]:
#-----Define important variables.

p_max = 1.8;
t_0   = 7.2; 
t_1   = 7.4;
steps_ramp = 5e4

In [8]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [9]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 4.5, tauP = 4.5, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [10]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 737205 / 787000 | TPS 20.463 | ETA 00:40:33
Time 00:00:20 | Step 737417 / 787000 | TPS 21.1686 | ETA 00:39:02
Time 00:00:30 | Step 737637 / 787000 | TPS 21.9839 | ETA 00:37:25
Time 00:00:40 | Step 737864 / 787000 | TPS 22.6353 | ETA 00:36:10
Time 00:00:50 | Step 738087 / 787000 | TPS 22.2486 | ETA 00:36:38
Time 00:01:00 | Step 738326 / 787000 | TPS 23.8227 | ETA 00:34:03
Time 00:01:10 | Step 738551 / 787000 | TPS 22.3401 | ETA 00:36:08
Time 00:01:20 | Step 738786 / 787000 | TPS 23.3371 | ETA 00:34:25
Time 00:01:30 | Step 739024 / 787000 | TPS 23.775 | ETA 00:33:37
Time 00:01:40 | Step 739252 / 787000 | TPS 22.7939 | ETA 00:34:54
Time 00:01:50 | Step 739484 / 787000 | TPS 23.1875 | ETA 00:34:09
Time 00:02:00 | Step 739713 / 787

In [11]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x7f706423f9e8>),
             ('particles', <hoomd.data.particle_data at 0x7f7064017710>),
             ('number_density', 0.4596255257875519),
             ('bonds', <hoomd.data.bond_data at 0x7f70640177b8>),
             ('angles', <hoomd.data.angle_data at 0x7f7064017828>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x7f7064017898>),
             ('impropers', <hoomd.data.dihedral_data at 0x7f7064017908>),
             ('constraints', <hoomd.data.constraint_data at 0x7f7064017978>),
             ('pairs', <hoomd.data.bond_data at 0x7f70640179e8>),
             ('timestep', 787000)])

In [12]:
#-----Get volume.

system.box.get_volume()

19589.860647038626